In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier

import xgboost, lightgbm
from mlxtend.classifier import EnsembleVoteClassifier

import warnings
warnings.filterwarnings("ignore")

In [30]:
training_df = pd.read_csv('../UNSW_NB15/UNSW_NB15_training-set.csv')
testing_df = pd.read_csv('../UNSW_NB15/UNSW_NB15_testing-set.csv')
combined_data = pd.concat([training_df, testing_df]).drop(['id'],axis=1)

combined_data.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
1,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
3,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
4,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


In [8]:
# Determine the categorical and numerical columns
categorical_columns = combined_data.select_dtypes(include=['object']).columns
print(f'The categorical columns are: {categorical_columns} \n')

# Determine how many unique values are in each categorical column
for column in categorical_columns:
    print(f'The column {column} has {combined_data[column].nunique()} unique values')

# Convert the categorical columns to numerical
le = LabelEncoder()
for column in categorical_columns:
    combined_data[column] = le.fit_transform(combined_data[column])

print(combined_data.head())

The categorical columns are: Index(['proto', 'service', 'state', 'attack_cat'], dtype='object') 

The column proto has 133 unique values
The column service has 13 unique values
The column state has 11 unique values
The column attack_cat has 10 unique values
        dur  proto  service  state  spkts  dpkts  sbytes  dbytes         rate  \
0  0.000011    119        0      5      2      0     496       0   90909.0902   
1  0.000008    119        0      5      2      0    1762       0  125000.0003   
2  0.000005    119        0      5      2      0    1068       0  200000.0051   
3  0.000006    119        0      5      2      0     900       0  166666.6608   
4  0.000010    119        0      5      2      0    2126       0  100000.0025   

   sttl  ...  ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  ct_ftp_cmd  \
0   254  ...                 1               2             0           0   
1   254  ...                 1               2             0           0   
2   254  ...               

In [23]:
# Use the train_test_split function to split the data into training and testing sets
X = combined_data.drop(['label'], axis=1)
y = combined_data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(206138, 43) (51535, 43) (206138,) (51535,)


In [20]:
# Run a grid search to find the best hyperparameters
parameter_space = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (150, 100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'learning_rate_init': [0.001, 0.01, 0.1]
}

mlp = MLPClassifier(max_iter=100)
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

print("Best parameters found:\n", clf.best_params_)

Best parameters found:
 {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.1, 'solver': 'sgd'}


In [21]:
# Train the model using the best hyperparameters
model = MLPClassifier(hidden_layer_sizes=(100,),
                      activation='relu',
                      solver='sgd',
                      learning_rate_init= 0.1,
                      max_iter=100)

model.fit(X_train, y_train)

# Evaluate the model, turn the predictions into a numpy array
y_pred = model.predict(X_test)
y_pred = np.array(y_pred)

y_test = np.array(y_test)

accuracy = np.mean(y_pred == y_test)
print(f'The accuracy of the model is: {accuracy}')



The accuracy of the model is: 0.9999223828466091


In [33]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1, verbosity=-1) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train,y_train)
    pred = clf.score(X_test,y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 1.00000 for the RandomForestClassifier
Acc: 0.99971 for the ExtraTreesClassifier
Acc: 1.00000 for the XGBClassifier
Acc: 1.00000 for the LGBMClassifier
